In [1]:
%matplotlib inline

In [11]:
from sentence_transformers import SentenceTransformer, models, losses, InputExample

In [17]:
from torch.utils.data import Dataset, DataLoader

In [19]:
from lightning import LightningModule, LightningDataModule

In [4]:
# Initialize the embedding model with a pre-trained Transformer
word_embedding_model = models.Transformer('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
# Initialize the pooling model to aggregate token embeddings into a single sentence embedding
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

In [15]:
# Combine them into a SentenceTransformer model
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [18]:
class SentenceDataset(Dataset):
    def __init__(self, sentences):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        # For simplicity, each item is just a sentence here. Adapt as needed.
        return InputExample(texts=[self.sentences[idx]])

In [ ]:
class SentenceDataModule(pl.LightningDataModule):
    def __init__(self, sentences, batch_size=32):
        super().__init__()
        self.sentences = sentences
        self.batch_size = batch_size

    def setup(self, stage=None):
        # Split your dataset here if necessary
        # For demonstration, we're treating all sentences as part of a single dataset
        self.dataset = SentenceDataset(self.sentences)

    def train_dataloader(self):
        return DataLoader(self.dataset, batch_size=self.batch_size, shuffle=True)


In [ ]:
class SentenceEmbeddingsModel(LightningModule):
    def __init__(self, model_name='bert-base-uncased', learning_rate=2e-5):
        super().__init__()
        # Using a pre-trained model from Hugging Face and adapting it for sentence embeddings
        word_embedding_model = models.Transformer(model_name)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        self.model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
        self.learning_rate = learning_rate

    def forward(self, sentences):
        return self.model.encode(sentences, convert_to_tensor=True)

    def training_step(self, batch, batch_idx):
        sentences = [example.texts[0] for example in batch]
        embeddings = self.forward(sentences)
        # Define your loss function here. For simplicity, this part is abstracted.
        loss = torch.tensor(0)  # Placeholder for actual loss calculation
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate)
        return optimizer


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from transformers import AdamW

class SentenceClassificationModel(pl.LightningModule):
    def __init__(self, sentence_classifier):
        super(SentenceClassificationModel, self).__init__()
        self.sentence_classifier = sentence_classifier

    def forward(self, input_ids, attention_mask, labels):
        logits = self.sentence_classifier(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(logits, labels)
        return loss, logits

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        loss, logits = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)

# # Assuming `sentence_classifier` is an instance of `SentenceClassifier` with your model
# model = SentenceClassificationModel(sentence_classifier)

# # Prepare your data loaders
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# # Train the model
# trainer = pl.Trainer(max_epochs=3, gpus=1 if torch.cuda.is_available() else 0)
# trainer.fit(model, train_loader)
